1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
    - 토큰 수 초과로 답변을 생성하지 못할 수 있음
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래 걸림
3. 임베딩 -> 벡터 DB에 저장한다
4. 사용자의 질문에 대해서 벡터 DB에서 유사도가 높은 문서를 찾는다
5. 유사도가 높은 문서를 LLM 에 질문과 함께 전달

In [ ]:
%pip install docx2txt langchain-community langchain-text-splitters

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)

loader = Docx2txtLoader("./tax.docx")
documents = loader.load_and_split(text_splitter)

print(len(documents))

187


In [ ]:
%pip install langchain-chroma

In [2]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

load_dotenv()

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

vector_store = Chroma.from_documents(
    documents = documents, 
    embedding = embeddings,
    persist_directory = "./chroma",
    collection_name = "tax",
)

In [3]:
from langchain_chroma import Chroma

vector_store = Chroma(
    embedding_function = embeddings,
    persist_directory = "./chroma",
    collection_name = "tax",
)

query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"

retrieved_docs = vector_store.similarity_search(query)

print(retrieved_docs)

[Document(id='7bb08e42-ad5a-499b-9914-2a886f49531c', metadata={'source': './tax.docx'}, page_content='1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 제외한다) 또는 해당 과세기간에 대통령령으로 정하는 총수입금액의 합계액이 2천만원 이하인 자의 주택임대소득(2018년 12월 31일 이전에 끝나는 과세기간까지 발생하는 소득으로 한정한다). 이 경우 주택 수의 계산 및 주택임대소득의 산정 등 필요한 사항은 대통령령으로 정한다.\n\n다. 대통령령으로 정하는 농어가부업소득\n\n라. 대통령령으로 정하는 전통주의 제조에서 발생하는 소득\n\n마. 조림기간 5년 이상인 임지(林地)의 임목(林木)의 벌채 또는 양도로 발생하는 소득으로서 연 600만원 이하의 금액. 이 경우 조림기간 및 세액의 계산 등 필요한 사항은 대통령령으로 정한다.\n\n바. 대통령령으로 정하는 작물재배업에서 발생하는 소득\n\n사. 대통령령으로 정하는 어로어업 또는 양식어업에서 발생하는 소득\n\n3. 근로소득과 퇴직소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 대통령령으로 정하는 복무 중인 병(兵)이 받는 급여\n\n나. 법률에 따라 동원된 사람이 그 동원 직장에서 받는 급여\n\n다. 「산업재해보상보험법」에 따라 수급권자가 받는 요양급여, 휴업급여, 장해급여, 간병급여, 유족급여, 유족특별급여, 장해특별급여, 장의비 또는 근로의 제공으로 인한 부상ㆍ질병ㆍ사망과 관련하여 근로자나 그 유족이 받는 배상ㆍ보상 또는 위자(慰藉)의 성질이 있는 급여\n\n라. 「근로기준법」 또는 「선원법」에 따라 근로자ㆍ선원 및 그 유족이 받는 요양보상금, 휴업보상금

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다
- [Context]를 참고해서 사용자의 질문에 답변해주세요

[Context]
{retrieved_docs}

[Question]
{query}
"""

response = llm.invoke(prompt)

print(response.content)

연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 다음의 단계를 고려해야 합니다.

1. **소득공제**: 직장인은 기본공제, 인적공제, 기타 소득공제를 적용받을 수 있습니다. 기본공제는 대개 150만원, 인적공제는 본인의 경우 150만원, 배우자 및 자녀에 대해 추가 공제를 받을 수 있습니다.

2. **세율 적용**: 한국의 소득세는 누진세로, 과세표준에 따라 세율이 달라집니다. 2023년 기준 세율은 다음과 같습니다:
   - 1,200만원 이하: 6%
   - 1,200만원 초과 ~ 4,600만원: 15%
   - 4,600만원 초과 ~ 8,800만원: 24%

3. **과세표준 계산**: 
   - **연봉**: 5,000만원
   - **공제 금액**: 예를 들어 기본공제와 인적공제를 모두 합쳐 300만원으로 가정한다면,
   - **과세표준**: 5,000만원 - 300만원 = 4,700만원 

4. **세액 계산**: 
   - 1,200만원 이하: 1,200만원 × 6% = 72만원
   - 1,200만원 초과 ~ 4,600만원: (4,600만원 - 1,200만원) × 15% = 5,400만원 × 15% = 810만원
   - 4,600만원 초과 ~ 4,700만원: (4,700만원 - 4,600만원) × 24% = 100만원 × 24% = 24만원

   따라서 총 세액은 72만원 + 810만원 + 24만원 = 906만원 입니다.

5. **세액공제**: 추가적으로 세액공제를 받을 수 있는 경우가 있는데, 예를 들어 근로소득세액공제 등을 적용받으면 세액이 줄어들 수 있습니다.

이와 같이 계산된 소득세는 대략 906만원이지만, 세액공제를 고려하지 않았으므로 최종 세액은 더 낮을 수 있습니다. 정확한 세액 계산은 개인의 상황에 따라 차이가 발생하므로 세무 전문가와 상담하는 것이 좋습니다.


In [ ]:
%pip install langchain langchainhub

In [14]:
from langchain_openai import ChatOpenAI
from langchain import hub

load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt = hub.pull("rlm/rag-prompt")
print(prompt)

/opt/anaconda3/envs/llm-application/lib/python3.10/site-packages/langsmith/client.py:280: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [9]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    retriever = vector_store.as_retriever(),
    chain_type_kwargs = {"prompt": prompt}
)

message = qa_chain.invoke({"query": query})

print(message)

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?', 'result': '죄송하지만, 제공된 정보에는 연봉 5천만원인 직장인의 소득세에 대한 구체적인 내용이 포함되어 있지 않습니다. 따라서 정확한 소득세 금액을 알려드릴 수 없습니다.'}
